In [22]:
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Standardscaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator , TransformerMixin

from sklearn.linear_model import LinearRegression

In [ ]:
class CombinedAttributesAdder(BaseEstimator , TransformerMixin):
    def fit(self , X , y=None):
        
        


In [14]:
x_train = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\ScaledTrainSet_data.csv' , index_col  = 'index')
x_train

,year,is_GAC,is_maserati,is_daihatsu,is_porsche,is_ds,is_smart,is_seres,is_cadillac,is_דודג',...,is_leasing,is_company,is_private,next_test_in,annual_licensing_fee,car_length,large,small,mini,medium
index,,,,,,,,,,,,,,,,,,,,,
2902,0.857143,0,0,0,0,0,0,0,0,0,...,0,0,1,0.500000,0.645547,0.562429,0,1,0,0
73,0.857143,0,0,0,0,0,0,0,0,0,...,0,0,1,0.833333,0.295767,0.798861,1,0,0,0
1537,0.809524,0,0,0,0,0,0,0,0,0,...,1,0,0,0.583333,0.188003,0.673624,0,0,0,1
2638,0.761905,0,0,0,0,0,0,0,0,0,...,1,0,0,0.166667,0.188003,0.592030,0,0,0,1
621,0.809524,0,0,0,0,0,0,0,0,0,...,0,0,1,0.416667,0.122825,0.588994,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,0.904762,0,0,0,0,0,0,0,0,0,...,1,0,0,0.416667,0.645547,0.798861,1,0,0,0
1095,0.476190,0,0,0,0,0,0,0,0,0,...,0,0,1,0.000000,0.219683,0.733966,0,0,0,1
1130,0.809524,0,0,0,0,0,0,0,0,0,...,1,0,0,0.666667,0.188003,0.732827,0,0,0,1


# Cross-validation :

In [ ]:
#first we return the demi-model (the average) and try to be better than him.... 

In [17]:
x_target = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\TrainSet-target_data.csv' , index_col='index' )
x_target.mean() 
demi-model : return  111097.128101

price    111097.128101
dtype: float64

In [34]:
x_target[["price"]] = MinMaxScaler().fit_transform(x_target[["price"]]) 
x_target = x_target["price"]

In [47]:
lin_reg =  LinearRegression()
lin_reg = lin_reg.fit(x_train, x_target)


#y_pred = lin_reg.predict(x)
# scoring= 'mean_squared_error'
print (cross_val_score(lin_reg, x_train, x_target, cv=3))


[-9.18231336e+21  7.92199796e-01 -1.45063363e+20]


In [ ]:
ולהבין כמה המודל שלנו צריך לתת ... דמי מודל- לבדוק מה המודל 0
lin_reg =  LinearRegression()
לפצל שוב את הטריין ולהשתמש בחלק השני לפרדיקט או משהו כזה 
lin_reg.fit(X_train, y_train)
בשלב האימון מודל אני יכולה לפצל שוב את הטריין וטסט  